In [ ]:
import torch
import numpy as np
import openmm
import openmm.app as app
import openmm.unit as u
import bisect
from rdkit.Chem import TorsionFingerprints
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Geometry import Point3D

import sys
sys.path.append("main")
import pickle
import main.utils

In [3]:
def np_to_mm(arr: np.ndarray, unit: openmm.unit=u.angstrom):
    wrapped_val = openmm.unit.quantity.Quantity(arr, unit)
    return wrapped_val

In [4]:
import sys

# OpenMM Imports
import simtk.openmm as mm
import simtk.openmm.app as app

# ParmEd Imports
from parmed.charmm import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet
from parmed.openmm import StateDataReporter
from parmed import unit as u

# Load the CHARMM files
print('Loading CHARMM files...')
toppar = [
    "toppar/par_all36_prot.prm", 
    "toppar/top_all36_prot.rtf",
    "toppar/toppar_water_ions.str",
]

params = CharmmParameterSet(*toppar)
ala5_gas = CharmmPsfFile('1uao_8_autopsf.psf')

system = ala5_gas.createSystem(params, nonbondedMethod=app.NoCutoff,
                               constraints=app.HBonds, implicitSolvent=app.HCT,
                               implicitSolventSaltConc=0.1 * u.moles/u.liter
)

# Create the integrator to do Langevin dynamics
integrator = mm.LangevinIntegrator(
                        300 * u.kelvin,       # Temperature of heat bath
                        1.0 / u.picoseconds,  # Friction coefficient
                        2.0 * u.femtoseconds, # Time step
)

# Define the platform to use; CUDA, OpenCL, CPU, or Reference. Or do not specify
# the platform to use the default (fastest) platform
platform = mm.Platform.getPlatformByName('CUDA')
prop = dict(CudaPrecision='mixed') # Use mixed single/double precision

# Create the Simulation object
sim = app.Simulation(ala5_gas.topology, system, integrator, platform, prop)

/home/yppatel/anaconda3/envs/myenv/lib/python3.8/site-packages/parmed/charmm/parameters.py:880: UserWarning: WARNING: Ignoring "DELETE ACCE NE2" because entity type ACCE not used.
  warnings.warn('WARNING: Ignoring "%s" because entity type %s not '


Loading CHARMM files...


In [5]:
ala5_crds = app.PDBFile('1uao_12_autopsf.pdb')

sim.context.setPositions(ala5_crds.positions)
energy = sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)    
print(f"before: {energy}")
sim.minimizeEnergy(maxIterations=2000)
energy = sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)    
print(f"after: {energy}")

before: 32.464435790823984 kcal/mol
after: -237.72226764762001 kcal/mol


In [6]:
pdb = "/home/yppatel/misc/clean_idp_rl/disordered_mol_untrained/mol0.pdb"
mol = Chem.rdmolfiles.MolFromPDBFile(pdb, removeHs=False)
AllChem.EmbedMultipleConfs(mol, numConfs=1000, numThreads=-1)

[22:00:43] Molecule does not have explicit Hs. Consider calling AddHs()


In [ ]:
from rdkit.Geometry import Point3D

In [ ]:
energies = []
for i in range(mol.GetNumConformers()):
    pos_a = np_to_mm(mol.GetConformer(i).GetPositions())
    sim.context.setPositions(pos_a)
    try:
        sim.minimizeEnergy(maxIterations=1000)
    except:
        energies.append(float("inf"))
        continue

    optimized_positions_nm = sim.context.getState(getPositions=True).getPositions()
    optimized_positions = optimized_positions_nm.in_units_of(u.angstrom) # match RDKit/MMFF convention

    conf = mol.GetConformer(i)
    for j, pos in enumerate(optimized_positions):
        conf.SetAtomPosition(j, Point3D(pos.x, pos.y, pos.z))

    energy = sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)
    energies.append(energy)

In [17]:
clean_energies = [energy._value if not isinstance(energy, float) else energy for energy in energies]

In [18]:
sort = np.argsort(clean_energies)  # sort by increasing energy

new = Chem.Mol(mol)
new.RemoveAllConformers()

for i in sort:
    conf = mol.GetConformer(int(i))
    new.AddConformer(conf, assignId=True)

In [21]:
pos_a = np_to_mm(new.GetConformer(4).GetPositions())
sim.context.setPositions(pos_a)
sim.minimizeEnergy(maxIterations=1000)
sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)

Quantity(value=-235.45845195530066, unit=kilocalorie/mole)

In [ ]:
Chem.rdmolfiles.MolToPDBFile(new, "new.pdb")